In [11]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
from scipy.integrate import odeint
from mpl_toolkits.mplot3d import Axes3D
from PIL import Image
import IPython.display as IPdisplay

save_folder = 'images/animation'
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

# Définir l'état initial du système (c'est-à-dire les positions x, y, z dans l'espace)
etat_initial = [0.1,0.0,0.0]

# Définir les points temporels à résoudre, uniformément espacés entre les heures de début et de fin
tps_deb = 1
tps_fin=60
intervalle=100
tps_points = np.linspace(tps_deb, tps_fin, tps_fin *intervalle)

def lorenz(etat, t, sigma=10, rho=28, beta=2.667):
    x, y, z = etat
    deriv_x = sigma * (y - x)
    deriv_y = x * (rho - z) - y
    deriv_z = x * y - beta * z
    return [deriv_x, deriv_y, deriv_z]

def diviser_en_morceaux(liste, taille):
    taille = max(1, taille)
    morceaux = [liste[0:i] for i in range(1, len(liste) + 1, taille)]
    return morceaux

morceaux = diviser_en_morceaux(tps_points,taille=20)

# Obtenir les points à tracer, un morceau de pas de temps à la fois, en intégrant le système d'équations
points = [odeint(lorenz,etat_initial,morceau) for morceau in morceaux]

# Tracer chaque ensemble de points, un à la fois, en enregistrant chaque tracé
fig = plt.figure(figsize=(13,9))
ax = fig.add_subplot(111, projection='3d')

for n, point in enumerate(points):
    ax.clear() # Effacer le contenu précédent de l'axe
    ax.xaxis.set_pane_color((1,1,1,1))
    ax.yaxis.set_pane_color((1,1,1,1))
    ax.zaxis.set_pane_color((1,1,1,1))
    x = point[:, 0]
    y = point[:, 1]
    z = point[:, 2]
    ax.plot(x, y, z, color='m', alpha=0.9,linewidth=0.9)
    ax.set_xlim((-40,40))
    ax.set_ylim((-40,40))
    ax.set_zlim((0,50))
    ax.set_title('Lorenz system attractor')
    
    plt.savefig('{}/{:03d}.png'.format(save_folder, n), dpi=60, bbox_inches='tight', pad_inches=0.1)

plt.close(fig)

# Charger toutes les images statiques dans une liste
images = [Image.open(image) for image in glob.glob('{}/*.png'.format(save_folder))]
gif_filepath = 'images/animation/lorenz_animation.gif'

# Créer un tuple de durées d'affichage, une pour chaque image
deb_fin=100
standard_durée = 5 
durées=(first_last,) + (standard_durée,) * (len(images) - 2) + (deb_fin,)


# Sauvegarde du GIF animé avec les durées correctes
for i in range(len(images)):
    images[i].info['duration'] = durées[i]

images[0].save(fp=gif_filepath, format='gif', save_all=True, append_images=images[1:])
# Vérifier que le nombre d'images dans le gif est égal au nombre de fichiers image et aux durées
IPdisplay.Image(url=gif_filepath)